In [105]:
from openpyxl import load_workbook

In [106]:
excel_path = "scheme.xlsx"
sheet_name = "Sheet1"
latex_path = "scheme.tex"

In [107]:
wb = load_workbook(excel_path)

In [108]:
def escape(s: str):
    return s.replace("_", "\\_")

In [109]:
def table_to_latex(workbook, sheet_name, headers: str | None=None):
    sheet = workbook[sheet_name]
    latex = ""

    if not headers:
        latex += "\\begin{tabular}{" + "l" * sheet.max_column + "}\n\\hline\n"
    else:
        latex += "\\begin{tabular}{" + headers + "}\n"
    
    latex += "\\toprule\n"

    for row in sheet.iter_rows():

        cells_latex = []
        cline_ranges = []

        for cell in row:

            # Find merged range
            is_merged, is_merged_master, is_merged_tail = False, False, False
            for merged_range in sheet.merged_cells.ranges:
                if cell.coordinate in merged_range:
                    is_merged = True
                    is_merged_master = is_merged and cell.value is not None
                    is_merged_tail = is_merged and cell.column == merged_range.max_col and cell.row == merged_range.max_row
                    break

            cell_value = escape(str(cell.value)) if cell.value else ""
            cell_value = f"\\textbf{{{cell_value}}}" if cell.font.bold else cell_value
            cell_value = f"\\underline{{{cell_value}}}" if cell.font.underline else cell_value
            cell_value = f"\\textit{{{cell_value}}}" if cell.font.italic else cell_value

            if is_merged and is_merged_master:
                merged_size_columns = merged_range.max_col - merged_range.min_col + 1
                merged_size_rows = merged_range.max_row - merged_range.min_row + 1
                cells_latex.append(
                    f"\\multirow{{{merged_size_rows}}}{{*}}{{{cell_value}}}"
                )

            elif is_merged:
                cells_latex.append("")
            else:
                cells_latex.append(f"{cell_value}")

            if not is_merged:
                cline_ranges.append(cell.column)
            elif is_merged_tail:
                cline_ranges.append(cell.column)

        latex += " & ".join(cells_latex) + " \\\\\n"
        latex += "".join(
            map(lambda x: f"\\cmidrule{{{str(x)}-{str(x)}}}", set(cline_ranges))
        )
        latex += "\n"

    latex += "\\bottomrule\n"
    latex += "\\end{tabular}\n\n"

    return latex

In [110]:
latex = ""
latex += table_to_latex(
    wb, "Sheet1", headers="p{1.6cm}p{1.7cm}p{2cm}p{3.3cm}p{7.5cm}"
)
latex += table_to_latex(
    wb, "Sheet2", headers="p{1.6cm}p{1.7cm}p{2cm}p{4.3cm}p{6.5cm}"
)

In [111]:
# Save the LaTeX code to a .tex file
with open(latex_path, "w") as f:
    f.write(latex)